In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import json
from bs4 import BeautifulSoup




In [6]:
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021','2022']

In [7]:
url = base_url+'/'+leagues[0]+'/'+seasons[8]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable, under 'script' tags
scripts = soup.find_all('script')
scripts


[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var datesData 	= JSON.parse('\x5B\x7B\x22id\x22\x3A\x2218962\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22157\x22,\x22title\x22\x3A\x22Osasuna\x22,\x22short_title\x22\x3A\x22OSA\x22\x7D,\x22a\x22\x3A\x7B\x22id\x22\x3A\x22138\x22,\x22title\x22\x3A\x22Sevilla\x22,\x22short_title\x22\x3A\x22SEV\x22\x7D,\x22goals\x22\x3A\x7B\x22h\x22\x3A\x222\x22,\x22a\x22\x3A\x221\x22\x7D,\x22xG\x22\x3A\x7B\x22h\x22\x3A\x221.35571\x22,\x22a\x22\x3A\x220.965574\x22\x7D,\x22datetime\x22\x3A\x222022\x2D08\x2D12\x2019\x3A00\x3A00\x22,\x22forecast\x22\x3A\x7B\x22w\x22\x3A\x220.465\x22,\x22d\x22\x3A\x220.3237\x22,\x22l\x22\x3A\x220.2113\x22\x7D\x7D,\x7B\x22id\x22\x3A\x2218963\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22152\x22,\x22title\x22\x3A\x22Celta\x20Vigo\x22,\x22short_title\x22\x3A\x22CEL\x22\x7D,\x22a\x22\x3A\x7B

In [8]:
string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in str(el):
      string_with_json_obj = str(el).strip()


#print(string_with_json_obj)
#print(string_with_json_obj.index("('"))

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
#print(ind_start)
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]



json_data = json_data.encode('utf8').decode('unicode_escape')
#print(json_data)
#json_data = json_data.encode('utf8').decode('unicode_escape')

type(json_data)

#print(json_data)
#json_data

str

In [9]:
data = json.loads(json_data)

#print(data)
type(data)

dict

In [10]:
data = json.loads(json_data)

print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

dict_keys(['138', '140', '141', '142', '143', '145', '146', '147', '148', '150', '152', '153', '154', '157', '207', '208', '223', '231', '239', '261'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 0.965574, 'xGA': 1.35571, 'npxG': 0.965574, 'npxGA': 0.612433, 'ppda': {'att': 142, 'def': 20}, 'ppda_allowed': {'att': 347, 'def': 22}, 'deep': 8, 'deep_allowed': 4, 'scored': 1, 'missed': 2, 'xpts': 0.9575999999999999, 'result': 'l', 'date': '2022-08-12 19:00:00', 'wins': 0, 'draws': 0, 'loses': 1, 'pts': 0, 'npxGD': 0.35314100000000004}


In [11]:
# Get teams and their relevant ids and put them into separate dictionary
data = json.loads(json_data)
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']


print(teams)
type(data)

{'138': 'Sevilla', '140': 'Real Sociedad', '141': 'Espanyol', '142': 'Getafe', '143': 'Atletico Madrid', '145': 'Rayo Vallecano', '146': 'Valencia', '147': 'Athletic Club', '148': 'Barcelona', '150': 'Real Madrid', '152': 'Celta Vigo', '153': 'Real Betis', '154': 'Villarreal', '157': 'Osasuna', '207': 'Elche', '208': 'Almeria', '223': 'Girona', '231': 'Real Valladolid', '239': 'Mallorca', '261': 'Cadiz'}


dict

In [12]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []

# Check the sample of values per each column
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']


In [13]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))
  
df = pd.DataFrame(sevilla_data, columns=columns)
df

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,0.965574,1.355710,0.965574,0.612433,"{'att': 142, 'def': 20}","{'att': 347, 'def': 22}",8,4,1,2,0.9576,l,2022-08-12 19:00:00,0,0,1,0,0.353141
1,h,1.951150,0.557629,1.951150,0.557629,"{'att': 159, 'def': 32}","{'att': 251, 'def': 21}",4,1,1,1,2.4412,d,2022-08-19 20:00:00,0,1,0,1,1.393521
2,a,1.378890,2.154070,1.378890,2.154070,"{'att': 134, 'def': 12}","{'att': 301, 'def': 22}",9,2,1,2,0.8148,l,2022-08-27 20:00:00,0,0,1,0,-0.775180
3,h,0.829157,4.678380,0.829157,4.678380,"{'att': 237, 'def': 31}","{'att': 202, 'def': 23}",8,8,0,3,0.0228,l,2022-09-03 19:00:00,0,0,1,0,-3.849223
4,a,2.320430,2.370550,2.320430,1.627280,"{'att': 217, 'def': 20}","{'att': 139, 'def': 23}",2,7,3,2,1.2948,w,2022-09-10 14:15:00,1,0,0,3,0.693150
5,a,0.436539,2.103580,0.436539,2.103580,"{'att': 179, 'def': 27}","{'att': 274, 'def': 24}",6,9,1,1,0.2375,d,2022-09-18 14:15:00,0,1,0,1,-1.667041


In [14]:
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

goals = dataframes['Real Sociedad']['scored']
goals.mean()
#goals.median()

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Osasuna.
Added data for Elche.
Added data for Almeria.
Added data for Girona.
Added data for Real Valladolid.
Added data for Mallorca.
Added data for Cadiz.


1.1666666666666667

In [15]:
#When we look at the content of DataFrame we can notice that such metrics as PPDA and OPPDA (ppda and ppda_allowed) are represented as total amounts of attacking/defensive actions, but in the original table it is shown as coefficients. Let’s fix that!
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  

In [16]:
#Now we have all our numbers, but for every single game. What we need is the totals for the team. Let’s find out the columns we have to sum up. For this we go back to original table at understat.com and we find that all metrics shoud be summed up and only PPDA and OPPDA are means in the end.
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef',]


In [17]:
#We are ready to calculate our totals and means. For this we loop through dictionary of dataframes and call .sum() and .mean() DataFrame methods that return Series, that’s why we add .transpose() to those calls. We put these new DataFrames into a list and after that concat them into a new DataFrame full_stat.

frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat

,team,matches,wins,draws,loses,scored,missed,pts,xG,npxG,xGA,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts
0,Sevilla,6,1.0,2.0,3.0,7.0,11.0,5.0,7.881740,7.881740,13.219919,11.733372,-3.851632,8.060035,11.274947,37.0,31.0,5.7687
0,Real Sociedad,6,3.0,1.0,2.0,7.0,8.0,10.0,10.685380,9.942100,8.176178,7.432904,2.509196,7.036200,12.859544,50.0,22.0,9.8545
0,Espanyol,6,1.0,1.0,4.0,7.0,12.0,4.0,8.344652,6.858108,10.016223,10.016223,-3.158115,14.121893,10.264868,30.0,44.0,7.2548
0,Getafe,6,2.0,1.0,3.0,6.0,12.0,7.0,5.181147,4.437873,9.519120,9.519120,-5.081247,14.578283,8.161227,9.0,37.0,5.2244
0,Atletico Madrid,6,3.0,1.0,2.0,10.0,6.0,10.0,9.320070,9.320070,7.926870,7.926870,1.393200,15.480277,12.496031,33.0,33.0,9.1654
0,Rayo Vallecano,6,2.0,1.0,3.0,7.0,8.0,7.0,6.147581,6.147581,10.661117,10.661117,-4.513536,9.351337,11.104345,25.0,32.0,5.9159
0,Valencia,6,3.0,0.0,3.0,10.0,5.0,9.0,10.229948,9.486668,6.040097,6.040097,3.446571,7.411713,11.859242,32.0,26.0,11.1142
0,Athletic Club,6,4.0,1.0,1.0,12.0,4.0,13.0,12.212207,10.725647,3.875478,3.875478,6.850169,11.271496,10.609815,50.0,23.0,13.3727
0,Barcelona,6,5.0,1.0,0.0,18.0,1.0,16.0,19.224350,19.224350,3.381053,3.381053,15.843297,6.462087,16.057673,76.0,19.0,15.7164
0,Real Madrid,6,6.0,0.0,0.0,17.0,6.0,18.0,15.273060,13.664960,6.142750,5.399474,8.265486,13.428608,18.942291,66.0,34.0,13.5602


In [18]:
#add mean for scored and received goals

full_stat['mean_scored'] = full_stat['scored']/full_stat['matches']
full_stat['mean_missed'] = full_stat['missed']/full_stat['matches']
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored','mean_scored','missed','mean_missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
pd.options.display.float_format = '{:,.2f}'.format
full_stat
  


,team,matches,wins,draws,loses,scored,mean_scored,missed,mean_missed,pts,xG,npxG,xGA,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts
0,Sevilla,6,1.00,2.00,3.00,7.00,1.17,11.00,1.83,5.00,7.88,7.88,13.22,11.73,-3.85,8.06,11.27,37.00,31.00,5.77
0,Real Sociedad,6,3.00,1.00,2.00,7.00,1.17,8.00,1.33,10.00,10.69,9.94,8.18,7.43,2.51,7.04,12.86,50.00,22.00,9.85
0,Espanyol,6,1.00,1.00,4.00,7.00,1.17,12.00,2.00,4.00,8.34,6.86,10.02,10.02,-3.16,14.12,10.26,30.00,44.00,7.25
0,Getafe,6,2.00,1.00,3.00,6.00,1.00,12.00,2.00,7.00,5.18,4.44,9.52,9.52,-5.08,14.58,8.16,9.00,37.00,5.22
0,Atletico Madrid,6,3.00,1.00,2.00,10.00,1.67,6.00,1.00,10.00,9.32,9.32,7.93,7.93,1.39,15.48,12.50,33.00,33.00,9.17
0,Rayo Vallecano,6,2.00,1.00,3.00,7.00,1.17,8.00,1.33,7.00,6.15,6.15,10.66,10.66,-4.51,9.35,11.10,25.00,32.00,5.92
0,Valencia,6,3.00,0.00,3.00,10.00,1.67,5.00,0.83,9.00,10.23,9.49,6.04,6.04,3.45,7.41,11.86,32.00,26.00,11.11
0,Athletic Club,6,4.00,1.00,1.00,12.00,2.00,4.00,0.67,13.00,12.21,10.73,3.88,3.88,6.85,11.27,10.61,50.00,23.00,13.37
0,Barcelona,6,5.00,1.00,0.00,18.00,3.00,1.00,0.17,16.00,19.22,19.22,3.38,3.38,15.84,6.46,16.06,76.00,19.00,15.72
0,Real Madrid,6,6.00,0.00,0.00,17.00,2.83,6.00,1.00,18.00,15.27,13.66,6.14,5.40,8.27,13.43,18.94,66.00,34.00,13.56


In [20]:
0.616**2


0.379456